pip install implicit

BUILDING AND EVALUATING A RECOMMENDER SYSTEM

This Jupyter Notebook explains how a error metric is generated for weighted matrix factorization. It will go through the data manipulation, splitting of the files, implementation of the different RS algorithms and the evaluation of these algorithms.




In [1]:
import random
import numpy as np
import pandas as pd
import time
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import operator
import time
import scipy as sp
import scipy.stats
import random
from sklearn import metrics
import implicit

LOADING THE DATA

In [2]:
#Please enter the path to the input data, which is available to download on XXXXXXXXXXXXX.
#Other input data should look like:
#0,1,3
#0,2,6
#.....
#where 0 is the user, 1 and 2 are items, and 3 and 6 are implicit feedback scores
#NOTE: Since this is python, the first item and user should both be labeled 0

filepath = 'C:/Users/peter/Documents/uvt/implicit/finalxdata.csv'


In [3]:
def load_matrix(filepath, num_users, num_items):
    counts = np.zeros((num_users, num_items))
    for i, line in enumerate(open(filepath, 'r')):
        user, item, count = line.strip().split(',')
        user = int(user)
        item = int(item)
        count = float(count)
        counts[user][item] = count
    return counts

# So what happens here?
# The formula creates an empty matrix (counts), of size U by I. 
# Then it starts stripping each row of the input file, where it extracts the user, item and feedback score.
# These values are then used to fill the empty matrix with the implicit feedback score.
# Finally it returns the matrix R. 


In [3]:
matrix1 = load_matrix(filepath, num_users = 6518, num_items = 4036)

NameError: name 'load_matrix' is not defined

So now we have loaded the matrix. Let's look at the row of a random user to get a feeling for the data(for this example we'll use user 100). 

In [5]:
item=0
for value in matrix1[99]:
    if value>0:
        print(item, value)
    item+=1

690 2.0
1244 2.0
1833 2.0
1882 3.0
1911 2.0
1929 2.0
1941 2.0
2774 2.0


So this user bought 7 different items 2 times, and 1 item 3 times. The code only prints the items on which the user has a purchase record, which means that the user did not purchase the other 4029 items. For good measure, let's look at user 200.

In [9]:
item=0
for value in matrix1[199]:
    if value>0:
        print(item, value)
    item+=1

139 2.0
1592 2.0
1806 2.0
1821 2.0
1870 2.0
1931 2.0
2070 2.0


The input data is ready, so lets get the train and test set in order. First, we'll need to select the users of which we will forget one interaction. The following function drops one value in a users row. 

In [7]:
# this function tells selects a random user-item-interactions that can be dropped
# its input is the users ID, and a user's implicit scores on all items
# it returns a list of the user ID, and the ID of the item that will be dropped by another function
def interactiontodrop(rownumber, row):
    scoreditems = []
    items = dict(enumerate(row))
    for k in items:
        if items[k] > 0:
            scoreditems.append(k)
    random.seed(1)
    if not scoreditems:
        print(rownumber)
    return [rownumber, random.choice(scoreditems)]

# this function 'drops' the interactions from the matrix (drop means set equal to zero)
# it checks which items can be dropped via the morethanx-function (currently all items pass the morethanx-function)
# it lists all users for whom items can be dropped, and then collects a sample of these users
# for each of these users a random item is selected by the interactiontodrop-function, whereafter it is dropped
# the input is a matrix, and the output is the list of user-item-interactions that are dropped,
# this list contains the values that are dropped by this function
def dropper(matrix, n):
    r = list(range(len(matrix)))
    random.seed(1)
    sample = random.sample(r,n)
    toodrop = []
    for x in sample:
        toodrop.append(interactiontodrop(x, matrix[x]))
    for item in toodrop:
        matrix[item[0]][item[1]] = 0
    return toodrop

In [9]:
# so let's check the functions, consider the following matrix X with three users, and four items.
X = [[1,0,0,1],
     [8,0,8,8],
     [9,9,9,9]]

# if we apply the dropper-function to this matrix, it will drop one value in each row, and replace it with zero,
# furthermore, it will remember the user and item id of the dropped interactions

print(dropper(X, 3))
print(X)

[[0, 0], [2, 1], [1, 0]]
[[0, 0, 0, 1], [0, 0, 8, 8], [9, 0, 9, 9]]


So in this example, we have dropped an interaction in each row, namely interaction [0,0], [2,1] and [1,0]
the original matrix X is now 
    X = [[0,0,0,1],
         [0,0,8,8],
         [9,0,9,9]]

With the dropper formula we can create the test set, which usually has the size of 20% of the data,
or in our case 20% of the 6518 users. Therefore, we call the dropper function on the input data with an N of 650.

In [11]:
test_set = dropper(matrix1, 650)
# so the test set is now ready
# and the first 10 user-item interactions in the test set are:
print(test_set[0:10])

[[1100, 590], [4662, 3351], [6256, 3907], [516, 615], [2089, 1817], [965, 449], [4058, 3394], [6233, 3485], [3682, 3417], [3868, 3376]]


with the test set in place, we can now train on the training data and generate recommendations
the code below fits a WMF model to the data
BTW, don't mind the function below, it is an artifact of my old thesis and I will develop a new error metric

In [1]:
# in order to evaluate our model we need to generate predictions for a user
# we generate predictions by multiplying the item and user vectors and then sort the items based on 
def prediction(userid, user_vectors, item_vectors, originalmatrix, n=4036):
    # the next line generates the score for a single user
    predictions = np.dot(user_vectors[userid], item_vectors.T)
    # since there are also predictions for items the user DID interact with, we need to set these items to zero
    # (of course these will score highly, since they have positive values in the original matrix)
    for i in range(4036):
        if originalmatrix[userid][i] > 0:
            predictions[i] = -6000
    #sort all items based on the score, and keep the sorted list of item-ids         
    dict = {key: value for (key, value) in (enumerate(predictions))}
    sorted_dict = sorted(dict.items(), key=operator.itemgetter(1), reverse=True)
    recommendation = []
    for i in range(n):
        recommendation.append(sorted_dict[i][0])
    return recommendation

def mpr_calc(dropped_items, user_vectors, item_vectors, originalmatrix):
    percentile_rankings = []
    #dropped item=test set
    #so for each user,item in the test set, generate the list for a user, locate the rank of an item (index(b[1]),
    #and then divide by the total number of items
    #finally store the ranking of this user in a list for all users
    for b in dropped_items:
        percentile_ranking = (prediction(b[0], user_vectors, item_vectors, originalmatrix).index(b[1]) + 1) / ((4036))
        percentile_rankings.append(percentile_ranking)
    return percentile_rankings

def sparser(originalmatrix, num_users, num_items):
    counts = sparse.dok_matrix((num_users, num_items), dtype=float)
    for i in range(len(originalmatrix)):
        row = originalmatrix[i]
        for j in range(len(row)):
            if row[j] > 0:
                user = int(i)
                item = int(j)
                count = float(row[j])
                counts[user, item] = count
    counts = counts.tocsr()
    return counts

In [2]:
matrix1 = matrix1*10
flipped = matrix1.T
sparsematrix = sparser(flipped, 4036, 6518)
wmf = implicit.als.AlternatingLeastSquares(factors=40, regularization=0.1, iterations=20)
wmf.fit(sparsematrix)
print(np.average(mpr_calc(test_set, (wmf.user_factors), (wmf.item_factors), matrix1)))

NameError: name 'matrix1' is not defined

In [22]:
# so now the model is fit and we can calculate with the recommendations
# furthermore we have acces to user and item factors
print(wmf.user_factors[1])
print(wmf.item_factors[1])

[ 0.75792477  0.83794315 -0.0441155   0.34144654  0.51893739 -0.0523536
  0.81810346 -0.08802933  0.7729655   0.55795031  0.30823591 -0.23916485
 -0.10338848  0.65232634 -0.11711397  0.71190949  0.15069208  0.23061752
  1.52397635 -0.18448975 -1.08876736 -1.18478348 -0.02331324 -0.70942825
  0.72819788  0.86041629  1.02140516  0.56453705 -1.74735187  0.35069892
  1.54398907  0.15716882  0.33339782  0.76760463 -0.21673031 -0.93450694
  0.30849883 -0.32614147  0.31815252 -0.67425088]
[ 0.01581864 -0.01425455 -0.00871603 -0.00455534  0.00651855  0.03813752
  0.00310788 -0.01560026  0.01996884 -0.03499781  0.0143725  -0.0178095
 -0.01310199 -0.03382893  0.01897174 -0.00755865  0.01592829  0.03693443
 -0.02923819  0.03252797  0.00131931  0.00615089  0.04650518 -0.02384858
  0.02067039 -0.0147373  -0.04415022  0.02059821 -0.03460832  0.02265688
  0.03844573  0.02502458  0.03888545  0.00633208  0.01160438 -0.00805943
  0.00858295  0.0191369  -0.01660148 -0.00780938]
-0.0190150277791
0.0677753

So in the example below, were we have user 2, and items 1, 2, and 3, we can see the score we give to the different items for user 1. As we can see, the user has the highest score on item 2, so we would recommend this item to this user. 

In [24]:
print(np.dot(wmf.item_factors[0],wmf.user_factors[1]))
print(np.dot(wmf.item_factors[1],wmf.user_factors[1])) #<-- the highest score
print(np.dot(wmf.item_factors[2],wmf.user_factors[1]))

-0.0190150277791
0.0677753535633
0.0244991572582
